# How to sample from a mesh?

In [2]:
from geomfum.dataset import NotebooksDataset
from geomfum.metric.mesh import (
    GraphShortestPathMetric,
    HeatDistanceMetric,
    VertexEuclideanMetric,
)
from geomfum.plot import MeshPlotter
from geomfum.sample import FarthestPointSampler, PoissonSampler, VertexProjectionSampler
from geomfum.shape import TriangleMesh


In [3]:
dataset = NotebooksDataset()
mesh = TriangleMesh.from_file(dataset.get_filename("cat-00"))

## "Coordinate" samplers

If we're interested in sampling coordinates of points on the mesh, we can use `PoissonSampler`.


In [ ]:
poisson_sampler = PoissonSampler.from_registry(min_n_samples=3, which="pymeshlab")
sampled_vertices = poisson_sampler.sample(mesh)
sampled_vertices

## Vertex indices samplers

If we're interested in sampling vertex indices instead, we can leverage "coordinate" samplers like `PoissonSampler` by projecting the coordinates to the closest vertices.

In [ ]:
vertex_sampler = VertexProjectionSampler(sampler=poisson_sampler)
sampled_indices = vertex_sampler.sample(mesh)
sampled_indices.astype(int)

To get vertex coordinates.

In [ ]:
sampled_vertices = mesh.vertices[sampled_indices]
sampled_vertices

Let's visualize the sampled points

In [ ]:
plotter = MeshPlotter.from_registry(colormap="RdBu", which="plotly")
plotter.add_mesh(mesh)
plotter.highlight_vertices(sampled_vertices)
plotter.show()

## Farthest point sampler

Alternatively, we can sample vertex indices directly with `FarthestPointSampler`. 

Select the metrics.

In [8]:
metrics = {
    "euclidean": VertexEuclideanMetric,
    "heat-distance": HeatDistanceMetric,
    "graph": GraphShortestPathMetric,
}

The vertex **7181** corresponds to the extreme of the cat’s tail. We start to sample from this point.


In [ ]:
fps = {}
samp = FarthestPointSampler(6,)

for name, metric in metrics.items():
    mesh.equip_with_metric(metric)
    samples = samp.sample(mesh, first_point= 7181)
    fps[name] = samples
    print(f"FPS {name}: {samples}")


Let's visualize the sampled points

In [ ]:
plotter = MeshPlotter.from_registry(colormap="RdBu", which="plotly")
plotter.add_mesh(mesh)
plotter.highlight_vertices(mesh.vertices[fps['euclidean']])
plotter.show()

In [ ]:
plotter = MeshPlotter.from_registry(colormap="RdBu", which="pyvista")
plotter.add_mesh(mesh)
plotter.highlight_vertices(mesh.vertices[fps['heat-distance']])
plotter.show()

In [ ]:
metric = list(fps.keys())[0]
plotter = MeshPlotter.from_registry(colormap="RdBu", which="polyscope")
plotter.add_mesh(mesh)
plotter.highlight_vertices(mesh.vertices[fps['graph']],)
#plotter.show()

### Sampling from a subset

The vertex **4265** corresponds to the extreme of the cat’s muzzle, i want to select the FPs on the tail.

In [13]:
points_on_the_tail = list(range(len(mesh.vertices) - 100, len(mesh.vertices)))

In [ ]:
fps = {}
dataset = NotebooksDataset()
samp = FarthestPointSampler(6,)
for name, metric in metrics.items():
    mesh.equip_with_metric(metric)
    samples = samp.sample(mesh, points_pool = points_on_the_tail, first_point= 4265)
    fps[name] = samples
    print(f"FPS {name}: {samples}")

In [ ]:
plotter = MeshPlotter.from_registry(colormap="RdBu", which="plotly")
plotter.add_mesh(mesh)
plotter.highlight_vertices(mesh.vertices[fps['euclidean']])
plotter.show()

## Further reading

* [How to compute descriptors?](./03_descriptors.ipynb)

* [How to set landmarks?](./06_landmarks.ipynb)